In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [2]:
for respond_perc in [.5]:
# for respond_perc in [.75, .25]:
    # for estate, include_state in [(True, True)]:
    for splitQ, splitM in [(True, True), (False, True), (True, False), (False, False)]:
        for estate, include_state, fullq in [(True, True, True)]:
            for fulls, insertpreds in [(False, True)]:
                for model, learning_rate, epochs in [("AdaptableLSTM", .07, 200),]:
                # for model, learning_rate, epochs in [("BasicNN", .0054, 500), ("LogisticRegressor", .003, 500)]:
                    for smooth, noise in [(0, .07)]:
                        for loss_fn in ["MSELoss"]:
                            test_metrics, train_metrics, adjusted_losses = [], [], []
                            for seed in range(3):
                                np.random.seed(seed)
                                torch.manual_seed(seed)
                                e = Experiment(
                                    numValFolds = 5,
                                    epochsToUpdateLabelMods = 10,
                                    data_kw={"minw": 2,
                                            "maxw": 29,
                                            "include_state": include_state,
                                            "include_pid": False,
                                            "expanded_states": estate,
                                            "top_respond_perc": respond_perc,
                                             "full_questionnaire": fullq,
                                             "full_sequence": fulls,
                                             "insert_predictions": insertpreds,
                                             "split_model_features": splitM,
                                             "split_weekly_questions": splitQ
                                            },
                                    model=model,
                                    model_kw={
                                        "lossfn": loss_fn,
                                        # "lossfn": "NDCG",
                                        # "lossfn": "CrossEntropyLoss",
                                        "hidden_size": 25, 
                                        "lr_step_mult": .9, 
                                        "lr_step_epochs": 60,
                                        "opt_kw": {
                                            "lr": learning_rate
                                        },
                                        "labelSmoothPerc": smooth,
                                        "gaussianNoiseStd": noise,
                                        "splitModel": splitM,
                                        "splitWeeklyQuestions": splitQ
                                    },
                                    train_kw={
                                        "epochs": epochs,
                                        "n_subj": 500,
                                        "rec_every": 5,
                                    })
                                # torch.autograd.set_detect_anomaly(True)
                                report = e.run()



                                individual_test_scores, labels = e.report_scores_individual_test()
                                individual_train_scores, labels = e.report_scores_individual_train()



                                dire = "./experiment_output/"
                                fileprefix = f"{model}LR{learning_rate}Resp{respond_perc}States{int(include_state)}Expanded{int(estate)}Seq{int(fulls)}Pred{int(insertpreds)}Smooth{smooth}Noise{noise}Split{int(splitQ)}{int(splitM)}"
                                np.savetxt(f"{dire}TRAINMETRICS-{fileprefix}S{seed}.csv", report["train_metrics"], delimiter = ',', header = ','.join(report['metric_labels']))
                                np.savetxt(f"{dire}TESTMETRICS-{fileprefix}S{seed}.csv", report["test_metrics"], delimiter = ',', header = ','.join(report['metric_labels']))
                                np.savetxt(f"{dire}IDVDTESTMETRICS-{fileprefix}S{seed}.csv", individual_test_scores, delimiter = ',', header = ','.join(report['metric_labels']))
                                np.savetxt(f"{dire}IDVDTRAINMETRICS-{fileprefix}S{seed}.csv", individual_train_scores, delimiter = ',', header = ','.join(report['metric_labels']))
                                np.savetxt(f"{dire}TRAINLOSSES-{fileprefix}S{seed}.csv", report["loss"], delimiter = ',')

                                preds1, preds2, preds3 = e.get_class_predictions(False)


                                np.savetxt(f"{dire}TRAINPREDS1-{fileprefix}S{seed}.csv", preds1, delimiter = ',')
                                np.savetxt(f"{dire}TRAINPREDS2-{fileprefix}S{seed}.csv", preds2, delimiter = ',')
                                np.savetxt(f"{dire}TRAINPREDS3-{fileprefix}S{seed}.csv", preds3, delimiter = ',')

                                plt.clf()
                                splot = plt.hist(preds1)
                                plt.title("Train Predictions for Class 1")
                                plt.savefig(f"{dire}/img/C1PredsTrain-{fileprefix}S{seed}.png")

                                plt.clf()
                                splot = plt.hist(preds2)
                                plt.title("Train Predictions for Class 2")
                                plt.savefig(f"{dire}/img/C2PredsTrain-{fileprefix}S{seed}.png")

                                plt.clf()
                                splot = plt.hist(preds3)
                                plt.title("Train Predictions for Class 3")
                                plt.savefig(f"{dire}/img/C3PredsTrain-{fileprefix}S{seed}.png")

                                preds1, preds2, preds3 = e.get_class_predictions(True)

                                np.savetxt(f"{dire}TESTPREDS1-{fileprefix}S{seed}.csv", preds1, delimiter = ',')
                                np.savetxt(f"{dire}TESTPREDS2-{fileprefix}S{seed}.csv", preds2, delimiter = ',')
                                np.savetxt(f"{dire}TESTPREDS3-{fileprefix}S{seed}.csv", preds3, delimiter = ',')

                                plt.clf()
                                splot = plt.hist(preds1)
                                plt.title("Test Predictions for Class 1")
                                plt.savefig(f"{dire}/img/C1PredsTest-{fileprefix}S{seed}.png")

                                plt.clf()
                                splot = plt.hist(preds2)
                                plt.title("Test Predictions for Class 2")
                                plt.savefig(f"{dire}/img/C2PredsTest-{fileprefix}S{seed}.png")

                                plt.clf()
                                splot = plt.hist(preds3)
                                plt.title("Test Predictions for Class 3")
                                plt.savefig(f"{dire}/img/C3PredsTest-{fileprefix}S{seed}.png")


                                plt.clf()
                                splot = plt.scatter(individual_test_scores[:, -1], individual_test_scores[:, labels.index("Acc")])
                                plt.title("Test Accuracy vs Response Count")
                                plt.savefig(f"{dire}/img/TestACCvResponse-{fileprefix}S{seed}.png")
                                plt.clf()
                                splot = plt.scatter(individual_train_scores[:, -1], individual_train_scores[:, labels.index("Acc")])
                                plt.title("Train Accuracy vs Response Count")
                                plt.savefig(f"{dire}/img/TrainACCvResponse-{fileprefix}S{seed}.png")
                                plt.clf()

                                splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
                                splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
                                splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
                                splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
                                plt.title("Train/Test Performance Over Training")
                                plt.legend()
                                plt.ylabel("Metric")
                                plt.xlabel("Training Epoch")
                                plt.savefig(f"{dire}/img/AccuracyVsEpoch-{fileprefix}S{seed}.png")
                                plt.clf()


                                writer = open(f"{dire}ALOSS-{fileprefix}.csv", "a")
                                writer.write(",".join([str(loss[0]) for loss in report["loss"]]))
                                writer.write("\n")
                                writer.close()

                                writer = open(f"{dire}FINALTRAINMETRICS-{fileprefix}.csv", "a")
                                writer.write(",".join([str(loss) for loss in report["train_metrics"][-1, :]]))
                                writer.write("\n")
                                writer.close()

                                writer = open(f"{dire}FINALTESTMETRICS-{fileprefix}.csv", "a")
                                writer.write(",".join([str(loss) for loss in report["test_metrics"][-1, :]]))
                                writer.write("\n")
                                writer.close()

                                Plotter.training_loss(report, dire)

                                plt.close("all")



['state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state' 'state'
 'state' 'state' 'state' 'state' 'state' 'state' 's

KeyboardInterrupt: 